# Análisis y Limpieza — `trending_by_time.csv`

Este notebook:
1. Carga el CSV detectando delimitador.
2. Estandariza nombres de columnas y elimina duplicados.
3. Detecta y convierte columnas de **fecha** a `datetime`.
4. Genera un EDA rápido (nulos, describe numéricas, top categorías) y lo guarda en CSVs.
5. Exporta un **CSV limpio** con los cambios.

> **Uso:** Coloca el archivo `trending_by_time.csv` en la misma carpeta que este notebook (o ajusta la ruta en la celda de configuración).

In [2]:
# ================== Configuración ==================
import os, re, csv
import pandas as pd
import numpy as np

# Ruta de entrada/salida (ajusta si es necesario)
INPUT_PATH = "trending_by_time.csv"   # Ruta al CSV original
OUTPUT_CSV = "trending_by_time_limpio.csv"
EDA_DIR = "eda_out"

os.makedirs(EDA_DIR, exist_ok=True)
print("INPUT_PATH:", INPUT_PATH)
print("OUTPUT_CSV:", OUTPUT_CSV)
print("EDA_DIR:", EDA_DIR)

INPUT_PATH: trending_by_time.csv
OUTPUT_CSV: trending_by_time_limpio.csv
EDA_DIR: eda_out


In [3]:
# ================== Utilidades ==================
def cargar_archivo(ruta_archivo: str) -> pd.DataFrame:
    """Carga CSV/Excel detectando delimitador si es CSV."""
    if not os.path.exists(ruta_archivo):
        raise FileNotFoundError(f"No encuentro el archivo: {ruta_archivo}")
    if ruta_archivo.lower().endswith(".csv"):
        with open(ruta_archivo, "r", encoding="utf-8") as f:
            muestra = f.read(2048)
            dialect = csv.Sniffer().sniff(muestra, delimiters=[",",";","|","\t"])
        return pd.read_csv(ruta_archivo, sep=dialect.delimiter)
    elif ruta_archivo.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(ruta_archivo)
    else:
        raise ValueError("Formato no soportado. Usa .csv o .xlsx/.xls")

def limpiar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    """Estandariza nombres: strip, lower, espacios->'_', quita acentos simples."""
    def normaliza(s):
        s = str(s).strip().lower()
        s = s.replace(" ", "_").replace("-", "_").replace("/", "_")
        for k,v in {"á":"a","é":"e","í":"i","ó":"o","ú":"u","ñ":"n"}.items():
            s = s.replace(k, v)
        s = re.sub(r"[^a-z0-9_]", "", s)
        s = re.sub(r"_+", "_", s)
        return s
    df = df.copy()
    df.columns = [normaliza(c) for c in df.columns]
    return df

def quitar_duplicados(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop_duplicates()

def detecta_columnas_fecha(df: pd.DataFrame):
    """Detecta posibles columnas de fecha por nombre y parseo de muestra."""
    candidatos = []
    patrones_nombre = ["fecha", "date", "time", "timestamp", "datetime"]
    for col in df.columns:
        if any(p in col.lower() for p in patrones_nombre):
            candidatos.append(col)
    for col in df.columns:
        if col in candidatos:
            continue
        muestra = df[col].dropna().astype(str).head(20)
        if len(muestra) == 0:
            continue
        parseados = pd.to_datetime(muestra, errors="coerce", dayfirst=False, infer_datetime_format=True)
        if parseados.notna().mean() >= 0.7:
            candidatos.append(col)
    # únicos preservando orden
    return list(dict.fromkeys(candidatos))

def convierte_a_fecha(df: pd.DataFrame, columnas=None) -> pd.DataFrame:
    df = df.copy()
    if columnas is None:
        columnas = detecta_columnas_fecha(df)
    for col in columnas:
        df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
    return df, columnas

def resumen_nulos(df: pd.DataFrame) -> pd.DataFrame:
    tipos = df.dtypes.astype(str).rename("dtype")
    nulos = df.isna().sum().rename("missing")
    pct = (df.isna().mean()*100).round(2).rename("missing_pct")
    return pd.concat([tipos, nulos, pct], axis=1).reset_index().rename(columns={"index":"columna"})

## 1) Cargar datos

In [5]:
df = cargar_archivo(INPUT_PATH)
print("Dimensiones iniciales:", df.shape)
display(df.head())

Dimensiones iniciales: (12343, 5)


,record_id,region,trending_date,category_title,videos_count
0,1,France,2017-11-14 00:00:00.000000,Autos & Vehicles,8
1,2,France,2017-11-15 00:00:00.000000,Autos & Vehicles,2
2,3,France,2017-11-16 00:00:00.000000,Autos & Vehicles,6
3,4,France,2017-11-17 00:00:00.000000,Autos & Vehicles,8
4,5,France,2017-11-18 00:00:00.000000,Autos & Vehicles,4


## 2) Limpieza básica

In [7]:
df = limpiar_columnas(df)
df = quitar_duplicados(df)
print("Dimensiones tras quitar duplicados:", df.shape)
display(pd.DataFrame({"columnas": df.columns}))

Dimensiones tras quitar duplicados: (12343, 5)


,columnas
0,record_id
1,region
2,trending_date
3,category_title
4,videos_count


## 3) Detectar y convertir fechas a `datetime`

In [9]:
cols_fecha = detecta_columnas_fecha(df)
print("Columnas detectadas como fecha:", cols_fecha)
df, cols_fecha = convierte_a_fecha(df, cols_fecha if cols_fecha else None)
display(df.head())

Columnas detectadas como fecha: ['trending_date']


/var/folders/xk/dd3d30ws6s92dmt2_1xtdqnr0000gn/T/ipykernel_2521/1864760532.py:46: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parseados = pd.to_datetime(muestra, errors="coerce", dayfirst=False, infer_datetime_format=True)
/var/folders/xk/dd3d30ws6s92dmt2_1xtdqnr0000gn/T/ipykernel_2521/1864760532.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parseados = pd.to_datetime(muestra, errors="coerce", dayfirst=False, infer_datetime_format=True)
/var/folders/xk/dd3d30ws6s92dmt2_1xtdqnr0000gn/T/ipykernel_2521/1864760532.py:46: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A st

,record_id,region,trending_date,category_title,videos_count
0,1,France,2017-11-14,Autos & Vehicles,8
1,2,France,2017-11-15,Autos & Vehicles,2
2,3,France,2017-11-16,Autos & Vehicles,6
3,4,France,2017-11-17,Autos & Vehicles,8
4,5,France,2017-11-18,Autos & Vehicles,4


## 4) EDA rápido y guardado de reportes

In [11]:
# Resumen de nulos/dtypes
res_nulos = resumen_nulos(df)
res_nulos.to_csv(os.path.join(EDA_DIR, "resumen_nulos.csv"), index=False, encoding="utf-8")
display(res_nulos.head())

# Describe numéricas
desc_num = df.select_dtypes(include=["number"]).describe().T.reset_index().rename(columns={"index":"columna"})
desc_num.to_csv(os.path.join(EDA_DIR, "describe_numericas.csv"), index=False, encoding="utf-8")
display(desc_num.head())

# Top categorías (primeras 10 por columna object)
filas = []
cats = df.select_dtypes(include=["object", "category"])
for col in cats.columns:
    vc = cats[col].value_counts(dropna=False).head(10)
    for val, cnt in vc.items():
        filas.append({"columna": col, "valor": None if pd.isna(val) else str(val), "conteo": int(cnt)})
top_cat = pd.DataFrame(filas)
top_cat.to_csv(os.path.join(EDA_DIR, "top_categorias.csv"), index=False, encoding="utf-8")
display(top_cat.head())

print("Archivos EDA guardados en carpeta:", EDA_DIR)

,columna,dtype,missing,missing_pct
0,record_id,int64,0,0.0
1,region,object,0,0.0
2,trending_date,datetime64[ns],0,0.0
3,category_title,object,0,0.0
4,videos_count,int64,0,0.0


,columna,count,mean,std,min,25%,50%,75%,max
0,record_id,12343.0,6172.000000,3563.261521,1.0,3086.5,6172.0,9257.5,12343.0
1,videos_count,12343.0,27.545167,29.793491,2.0,8.0,18.0,36.0,220.0


,columna,valor,conteo
0,region,Russia,2930
1,region,United States,2860
2,region,France,2774
3,region,India,2283
4,region,Japan,1496


Archivos EDA guardados en carpeta: eda_out


## 5) Guardar CSV final limpio

In [13]:
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ CSV limpio guardado en: {OUTPUT_CSV}")

✅ CSV limpio guardado en: trending_by_time_limpio.csv
